## Add, Modify and Retrieve data from OrientDB


## 1. Install the PyOrient package

In [ ]:
! pip install pyorient --user 

** <font color=blue>Now restart the kernel by choosing Kernel > Restart. </font> **

## 2. Import packages and libraries

In [ ]:
import pyorient, json, pandas as pd
import sys

## 3. Add your service credentials for Object Storage
* You must create Object Storage service on Bluemix. To access data in a file in Object Storage, you need the Object Storage authentication credentials. Insert the Object Storage authentication credentials as <i><b>credentials_1</b></i> in the following cell after removing the current contents in the cell. Rename the variable to <i><b>credentials_1</b></i> if the variable name is different. 

In [ ]:
# @hidden_cell
credentials_1 = {
  'auth_url':'',
  'project':'',
  'project_id':'',
  'region':'',
  'user_id':'',
  'domain_id':'',
  'domain_name':'',
  'username':'',
  'password':'',
  'container':'',
  'tenantId':'',
  'filename':''
}

## 4. Loading the Configuration and Data Files

### 4.1 Load the config.json from Object Storage

In [ ]:
from io import BytesIO  
import requests  
import json  
import pandas as pd

def get_data(credentials):  
    """This functions returns a StringIO object containing
    the file content from Bluemix Object Storage V3."""

    url1 = ''.join(['https://identity.open.softlayer.com', '/v3/auth/tokens'])
    data = {'auth': {'identity': {'methods': ['password'],
            'password': {'user': {'name': credentials['username'],'domain': {'id': credentials['domain_id']},
            'password': credentials['password']}}}}}
    headers1 = {'Content-Type': 'application/json'}
    resp1 = requests.post(url=url1, data=json.dumps(data), headers=headers1)
    resp1_body = resp1.json()
    for e1 in resp1_body['token']['catalog']:
        if(e1['type']=='object-store'):
            for e2 in e1['endpoints']:
                        if(e2['interface']=='public'and e2['region']=='dallas'):
                            url2 = ''.join([e2['url'],'/', credentials['container'], '/', credentials['filename']])
    s_subject_token = resp1.headers['x-subject-token']
    headers2 = {'X-Auth-Token': s_subject_token, 'accept': 'application/json'}
    resp2 = requests.get(url=url2, headers=headers2)
    return json.loads(resp2.content)



node_data = get_data(credentials_1)
print node_data

### 4.2. Load the IMDb movie data from Object Storage

In [ ]:
'''There are many rows and columns in the data that are empty. Hence, It is important to clean the data.
All the empty rows and columns are dropped from the dataframe using dropna() function of pandas.'''

imdb_df = imdb_df.dropna()
imdb_df.head(10)

##  5. Connect to OrientDB
* Uncomment the first line after entering the IP address of the Kubernetes cluster and the port.

In [ ]:
#client = pyorient.OrientDB(<ip-address-of-the-kubernetes-cluster>,<node-port mapped to port 2424 of orientDB>)
print(client)

# The session id username and password is global password that was set for orientDB
session_id = client.connect(node_data['username'], node_data['password'])

## 6. Utility functions
These methods return dataframes containing the existing information of the database based on a condition.
* check_if_class_present_or_not.
* check_if_person_already_present.
* check_if_already_present_movie.
* find_the_Cluster_id_of_a_class


In [ ]:
def check_if_class_present_or_not(classname):
    '''This function checks if a class is present or not'''
    
    name="will be replaced by query result name"
    query = "SELECT FROM ( SELECT expand( classes ) FROM metadata:schema ) WHERE name = " +'"'+ classname + '"'
    a = client.command(query)
    
    for k in a:
        name = k.name
    
    if(name == classname):
        return True
    else:
        return False
    
     
def check_if_person_already_present():
    '''This function checks if the person already exists.'''
    
    check_if_already_present = "select * from person"
    c = client.command(check_if_already_present)
    d=[]

    for name in c:
        d.append([name.name, name.role, name.fblikes])


    check_df = pd.DataFrame(list(d), columns=['name','role','fblikes'])
    return check_df


def check_if_already_present_movie():
    '''This function checks if the movie already exists.'''
    
    check_if_already_present = "select title , year from movie"
    c = client.command(check_if_already_present)
  
    d=[]

    for name in c:
        d.append([name.title, name.year])


    check_df_movie = pd.DataFrame(list(d), columns=['title','year'])
    return check_df_movie


def find_the_Cluster_id_of_a_class(classname):
    '''This function finds the Cluster ID of the class.'''
    
    find_the_Cluster_id_of_a_class = "SELECT defaultClusterId from (SELECT expand( classes ) FROM metadata:schema) where name = '" + classname + "'"
    c = client.command(find_the_Cluster_id_of_a_class)
    for ids in c:
        return ids.defaultClusterId




## 7. Core Functions
These are the core functions of the notebook performing operations on OrientDB:
* Create Database.
* Create node class with its properties as per defined in config.json.
* Create node class when no schema defined.
* Create edge class.
* Create vertices/nodes with movie data.
* Create relations between these nodes.
* Create vertices for usecases where schema is indefinte.
* Other insights like :
    * Most mentioned movie.
    * Movies with rating above 7.


In [ ]:
def createDatabase(node_data):
    '''This function the database if it does not already exist.'''
    
    if client.db_exists( node_data['Database_name'], pyorient.STORAGE_TYPE_MEMORY ):
        client.db_open(node_data['Database_name'],node_data['username'], node_data['password'])
        print "The Database " + node_data['Database_name'] + " "+ "has already been created and opened for use."
    else:        
        client.db_create( node_data['Database_name'], pyorient.DB_TYPE_GRAPH, pyorient.STORAGE_TYPE_MEMORY )
        print "The Database " + node_data['Database_name'] + " created and opened successfully"
        

def createNodeClass_withSchema(node_data):
    '''This function creates a Node Class with a schema.'''
    
    for class_name,value in node_data['vertex_class'].items():
        bool_result = check_if_class_present_or_not(class_name)
        if(not bool_result):
            command_to_create_node = "create class"+" "+ class_name +" "+ "extends V"
            cluster_id = client.command(command_to_create_node) 
            for property_name,value in node_data['vertex_class'][class_name].items():
                command_to_create_property= "create property"+ " "+ class_name +"." + property_name +" " +"IF NOT EXISTS " + value
                client.command(command_to_create_property) 
            print "The class " + class_name + " and its properties have been created successfully."
        else:
            print "The class " + class_name + " has been created already."

def createNodeClass_NoSchema(node_data):
    '''This function creates a Node Class with no schema.'''
    
    for class_name,value in node_data['vertex_class'].items():
        if(check_if_class_present_or_not(class_name)):
            command_to_create_node = "create class"+" "+ class_name +" "+ "extends V"
            cluster_id = client.command(command_to_create_node) 
            print "The class " + class_name + " has been created with cluster id " +cluster_id
        else:
            print "The class " + class_name + " has been created already."
            

def createEdgeClass(node_data):
    '''This function checks if the edge class is already present.'''
    
    for class_name,v in node_data['edge_class'].items():       
        if(not check_if_class_present_or_not(class_name)):
            command_to_create_edge_class = "create class"+" "+ class_name +" "+ "extends E"
            cluster_id = client.command(command_to_create_edge_class)
            print("The Edge class" +" " + class_name + " has been created successfully.")

            for key,val in node_data['edge_class'][class_name].items():
                command_to_create_property= "create property"+ " "+ class_name +"." + key +" " +"IF NOT EXISTS " + node_data['edge_class'][class_name][key]['Type']+" " + node_data['edge_class'][class_name][key]['Linked_Class']
                client.command(command_to_create_property)                 
        else:
             print("The Edge class" +" " + class_name + " already exists.")
            

def creating_records(imdb_df):
    '''Create records in the database if it does not exist.'''
    
    for index, row in imdb_df.iterrows():
        check_df = check_if_person_already_present()
        if(any(check_df.name == row["actor_1_name"])):
            print "Node "+row["actor_1_name"] +" is already present."
        else:
            command_to_create_actor_1_node_class_person  = "INSERT INTO person (name, fblikes, role) VALUES (" +'"' +row["actor_1_name"]+'"' + ','+ str(row["actor_1_facebook_likes"])+',' +'"' +'actor'+'"' + ")"
            client.command(command_to_create_actor_1_node_class_person)
            
        if(any(check_df.name == row["actor_2_name"])):
            print "Node "+row["actor_2_name"] +" is already present."
        else:
            command_to_create_actor_2_node_class_person  = "INSERT INTO person (name, fblikes, role) VALUES (" +'"' +row["actor_2_name"]+'"' + ','+ str(row["actor_2_facebook_likes"])+',' +'"' +'actor'+'"' + ")"
            client.command(command_to_create_actor_2_node_class_person)
            
        if(any(check_df.name == row["actor_3_name"])):
            print "Node "+row["actor_3_name"] +" is already present."
        else:
            command_to_create_actor_3_node_class_person  = "INSERT INTO person (name, fblikes, role) VALUES (" +'"' +row["actor_3_name"]+'"' + ','+ str(row["actor_3_facebook_likes"])+',' +'"' +'actor'+'"' + ")"
            client.command(command_to_create_actor_3_node_class_person)
            
        if(any(check_df.name == row["director_name"])):
            print "Node "+row["director_name"] +" is already present."
        else:
            command_to_create_director_node_class_person  = "INSERT INTO person (name, fblikes, role) VALUES (" +'"' +row["director_name"]+'"' + ','+ str(row["director_facebook_likes"])+',' +'"' +'director'+'"' + ")"
            client.command(command_to_create_director_node_class_person)
            
        command_to_create_movie  = "INSERT INTO movie (title, year, durationInMins, imdbRating, genre, plotKeywords, numCriticForReviews, movieFacebookLikes) VALUES (" +'"' +row["movie_title"]+'"' + ','+ str(row["title_year"])+','+str(row["duration"])+','+str(row["imdb_score"])+',' +'"'+row["genres"]+'"' +','+'"' +row["plot_keywords"]+'"'+','+str(row["num_critic_for_reviews"])+','+str(row["movie_facebook_likes"])+")"
        client.command(command_to_create_movie)
        
        

        
def createRelationships():
    '''This function creates relationships.'''  
    
    check_df = check_if_person_already_present()
    check_df_movie = check_if_already_present_movie()
    
    for index, row in imdb_df.iterrows():
        # Create an edge between actors_1 and actor_2.
        if((row["actor_1_name"]  in check_df.name.values) and (row["actor_2_name"]  in check_df.name.values)):
            command_to_create_edge_between_two_actors_1_and_2 = "create edge worked_with from (select from person where name = "+'"'+row["actor_1_name"]+'"'+ ")"+" " +"to (select from person where name = "+'"'+row["actor_2_name"]+'"'+")"
            client.command(command_to_create_edge_between_two_actors_1_and_2)            
        else:
               print "Edge cant be created because vertex is not present.", row["actor_1_name"], ",",row["actor_2_name"]  


        # Create an edge between actors_2 and actor_3.
        if((row["actor_2_name"]  in check_df.name.values) and  (row["actor_3_name"]  in check_df.name.values) ):
            command_to_create_edge_between_two_actors_2_and_3 = "create edge worked_with from (select from person where name = "+'"'+row["actor_2_name"]+'"'+ ")"+" " +"to (select from person where name = "+'"'+row["actor_3_name"]+'"'+")"
            client.command(command_to_create_edge_between_two_actors_2_and_3)            
        else:
               print "Edge cant be created because one or both the vertices is not present -",row["actor_2_name"], ",",row["actor_3_name"]


        # Create an edge between actors_3 and actor_1.
        if((row["actor_3_name"]  in check_df.name.values) and ( row["actor_1_name"]  in check_df.name.values )):
            command_to_create_edge_between_two_actors_3_and_1 = "create edge worked_with from (select from person where name = "+'"'+row["actor_3_name"]+'"'+ ")"+" " +"to (select from person where name = "+'"'+row["actor_1_name"]+'"'+")"
            client.command(command_to_create_edge_between_two_actors_3_and_1)            
        else:
               print "Edge cant be created because one or both the vertices is not present -",row["actor_2_name"], ",",row["actor_3_name"]

        # Create an edge between actors_1 and director.
        if((row["actor_1_name"]  in check_df.name.values) and  (row["director_name"] in check_df.name.values) ):
            command_to_create_edge_between_actor_1_and_director = "create edge worked_with from (select from person where name = "+'"'+row["director_name"]+'"'+ ")"+" " +"to (select from person where name = "+'"'+row["actor_1_name"]+'"'+")"
            client.command(command_to_create_edge_between_actor_1_and_director)
        else:
               print "Edge cant be created because one or both the vertices is not present -",row["actor_1_name"], ",",row["director_name"]

        # Create an edge between actors_2 and director.
        if((row["actor_2_name"]  in check_df.name.values) and ( row["director_name"] in check_df.name.values) ):
            command_to_create_edge_between_actor_2_and_director = "create edge worked_with from (select from person where name = "+'"'+row["director_name"]+'"'+ ")"+" " +"to (select from person where name = "+'"'+row["actor_2_name"]+'"'+")"
            client.command(command_to_create_edge_between_actor_2_and_director)
        else:
               print "Edge cant be created because one or both the vertices is not present -",row["actor_2_name"], ",",row["director_name"]

        # Create an edge between actors_3 and director.
        if((row["actor_3_name"]  in check_df.name.values) and ( row["director_name"] in check_df.name.values) ):
            command_to_create_edge_between_actor_3_director = "create edge worked_with from (select from person where name = "+'"'+row["director_name"]+'"'+ ")"+" " +"to (select from person where name = "+'"'+row["actor_3_name"]+'"'+")"
            client.command(command_to_create_edge_between_actor_3_director)
        else:
               print "Edge cant be created because one or both the vertices is not present -",row["actor_3_name"], ",",row["director_name"]

        # Create an edge between actors_1 and movie.
        if((row["actor_1_name"]  in check_df.name.values) and  (row["movie_title"] in check_df_movie.title.values)):
            command_to_create_edge_between_actors_1_and_movie = "create edge acted_in from (select from person where name = "+'"'+row["actor_1_name"]+'"'+ ")"+" " +"to (select from movie where title = "+'"'+row["movie_title"]+'"'+")"
            client.command(command_to_create_edge_between_actors_1_and_movie)
        else:
               print "Edge cant be created because one or both the vertices is not present -",row["actor_1_name"], ",",row["movie_title"]

        # Create an edge between actors_2 and movie.
        if((row["actor_2_name"]  in check_df.name.values) and  (row["movie_title"] in check_df_movie.title.values) ):
            command_to_create_edge_between_actors_2_and_movie = "create edge acted_in from (select from person where name = "+'"'+row["actor_2_name"]+'"'+ ")"+" " +"to (select from movie where title = "+'"'+row["movie_title"]+'"'+")"
            client.command(command_to_create_edge_between_actors_2_and_movie)
        else:
               print "Edge cant be created because one or both the vertices is not present -",row["actor_2_name"], ",",row["movie_title"]

        # Create an edge between actors_3 and movie.
        if((row["actor_3_name"]  in check_df.name.values) and  (row["movie_title"] in check_df_movie.title.values)):
            command_to_create_edge_between_actors_3_and_movie = "create edge acted_in from (select from person where name = "+'"'+row["actor_3_name"]+'"'+ ")"+" " +"to (select from movie where title= "+'"'+row["movie_title"]+'"'+")"
            client.command(command_to_create_edge_between_actors_3_and_movie)
        else:
               print "Edge cant be created because one or both the vertices is not present -",row["actor_3_name"], ",",row["movie_title"]

        

def creating_records_noschema(data):
    '''This function creates records with no schema.'''
    
    id = client.record_create(cluster_id, data)
    print "Record succesfully created with " + str(id)
        


def most_mentioned_movie():
    '''This function retrieves the most mentioned movies.'''
    
    a = client.command(('select max(movieFacebookLikes) from movie '))

    for max_num in a :
        max_num = max_num.max
    
    most_mentioned_movie_object = client.command('select title from movie where movieFacebookLikes = ' + str(max_num))
    
    return most_mentioned_movie_object    
        

def movie_with_imdb_rating_above_7():
    '''This function retrieves the movies with IMDb rating > 7.'''
    
    title = []
    a = client.command(('select title from movie where imdbRating > 7 '))
    for titles in a :
        title.append(titles.title)
    title_df = pd.DataFrame(list(title), columns=['title'])    
    return title_df
        

     

## 8. Perform operations on OrientDB

### 8.1 Create Database, Node classes, Edge classes, and Records

In [ ]:
# Create Database
print ""
print "Creating database..."
createDatabase(node_data)
print "Database creation successful."

# Create Node class with schema
print ""
print "Creating Node classes with schema..."
createNodeClass_withSchema(node_data)
print "Node classes with schema creation successful."

# Create Edge class
print ""
print "Creating Edge classes..."
createEdgeClass(node_data)
print "Edge classes creation successful."

# Create records
print ""
print "Creating records..."
creating_records(imdb_df)
print "Records creation successful."

# Create Relationships
print ""
print "Creating relationships..."
createRelationships()
print "Relationships creation successful."


### 8.2 Create the records for the scenario where there is no schema defined

In [ ]:
data = {
        'person': {
                "name": "John", 
                "role": "director",
                "fblikes": 400000.0,
                "born_in" : 1980
                },
        }
# Get cluster id
for key, value in data.iteritems():
        cluster_id = find_the_Cluster_id_of_a_class(key)
        print "Cluster Id:",cluster_id

# Create record that has no schema
creating_records_noschema(data)

### 8.3 Insights from OrientDB

In [ ]:
most_mentioned_movie = most_mentioned_movie()
print "Most mentioned movies"
for titles in most_mentioned_movie:
    print titles.title
print ""    

movie_with_imdb_rating_above_7 = movie_with_imdb_rating_above_7()
print "Movies with IMDb rating > 7:"
print movie_with_imdb_rating_above_7

## 9. Visualisation of results
You can open OrientDB studio and execute the queries in the graph editor of OrientDB to view the graph you have built.
Watch this video : https://www.youtube.com/watch?v=l-OVSjf-vk0&t=4s for OrientDB tutorial.